In [62]:
import requests 
import json
import time
import os
import pandas as pd
from sqlalchemy import engine as sql
from IPython import display
import ipywidgets as widget
import matplotlib.pyplot as plt
import math

In [8]:
data = requests.get('https://api.hh.ru/vacancies?text=java').json()
data

{'items': [{'id': '44706219',
   'premium': False,
   'name': 'Junior Java Developer',
   'department': None,
   'has_test': False,
   'response_letter_required': False,
   'area': {'id': '1', 'name': 'Москва', 'url': 'https://api.hh.ru/areas/1'},
   'salary': {'from': 90000, 'to': 140000, 'currency': 'RUR', 'gross': False},
   'type': {'id': 'open', 'name': 'Открытая'},
   'address': None,
   'response_url': None,
   'sort_point_distance': None,
   'published_at': '2021-06-12T11:39:02+0300',
   'created_at': '2021-06-12T11:39:02+0300',
   'archived': False,
   'apply_alternate_url': 'https://hh.ru/applicant/vacancy_response?vacancyId=44706219',
   'insider_interview': None,
   'url': 'https://api.hh.ru/vacancies/44706219?host=hh.ru',
   'alternate_url': 'https://hh.ru/vacancy/44706219',
   'relations': [],
   'employer': {'id': '193245',
    'name': 'INTECH',
    'url': 'https://api.hh.ru/employers/193245',
    'alternate_url': 'https://hh.ru/employer/193245',
    'logo_urls': {'origi

# Собираем данные по специальности

In [63]:

 
def getPage(page = 0):
    params = {
        'text': 'NAME:Разработчик', # Текст фильтра. В имени должно быть слово "Аналитик"
        'area': 1, # Поиск ощуществляется по вакансиям города Москва
        'page': page, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
    }
     
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    req.close()
    return data
 
for page in range(0, 20):
    jsObj = json.loads(getPage(page))
    nextFileName = './docs/pagination/{}.json'.format(len(os.listdir('./docs/pagination')))
    f = open(nextFileName, mode='w', encoding='utf8')
    f.write(json.dumps(jsObj, ensure_ascii=False))
    f.close()
    if (jsObj['pages'] - page) <= 1:
        break
     
    # Необязательная задержка, но чтобы не нагружать сервисы hh, оставим. 5 сек мы может подождать
    time.sleep(0.25)
     
print('Старницы поиска собраны')

Старницы поиска собраны


# Собираем данные по вакансии

In [65]:
 
# Получаем перечень ранее созданных файлов со списком вакансий и проходимся по нему в цикле 
for fl in os.listdir('./docs/pagination'):
     
    # Открываем файл, читаем его содержимое, закрываем файл
    f = open('./docs/pagination/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
     
    # Преобразуем полученный текст в объект справочника
    jsonObj = json.loads(jsonText)
     
    # Получаем и проходимся по непосредственно списку вакансий
    for v in jsonObj['items']:
         
        # Обращаемся к API и получаем детальную информацию по конкретной вакансии
        req = requests.get(v['url'])
        data = req.content.decode()
        req.close()
         
        # Создаем файл в формате json с идентификатором вакансии в качестве названия
        # Записываем в него ответ запроса и закрываем файл
        fileName = './docs/vacancies/{}.json'.format(v['id'])
        f = open(fileName, mode='w', encoding='utf8')
        f.write(data)
        f.close()
         
        time.sleep(0.25)
         
print('Вакансии собраны')

Вакансии собраны


# Собираем данные в  базу данных

In [66]:
# Создаем списки для столбцов таблицы vacancies
IDs = [] # Список идентификаторов вакансий
names = [] # Список наименований вакансий
descriptions = [] # Список описаний вакансий
 
# Создаем списки для столбцов таблицы skills
skills_vac = [] # Список идентификаторов вакансий
skills_name = [] # Список названий навыков
 
# В выводе будем отображать прогресс
# Для этого узнаем общее количество файлов, которые надо обработать
# Счетчик обработанных файлов установим в ноль
cnt_docs = len(os.listdir('./docs/vacancies'))
i = 0
 
# Проходимся по всем файлам в папке vacancies
for fl in os.listdir('./docs/vacancies'):
     
    # Открываем, читаем и закрываем файл
    f = open('./docs/vacancies/{}'.format(fl), encoding='utf8')
    jsonText = f.read()
    f.close()
     
    # Текст файла переводим в справочник
    jsonObj = json.loads(jsonText)
     
    # Заполняем списки для таблиц
    IDs.append(jsonObj['id'])
    names.append(jsonObj['name'])
    descriptions.append(jsonObj['description'])
     
    # Т.к. навыки хранятся в виде массива, то проходимся по нему циклом
    for skl in jsonObj['key_skills']:
        skills_vac.append(jsonObj['id'])
        skills_name.append(skl['name'])
     
    # Увеличиваем счетчик обработанных файлов на 1, очищаем вывод ячейки и выводим прогресс
    i += 1
    display.clear_output(wait=True)
    display.display('Готово {} из {}'.format(i, cnt_docs))
 
 

# Создаем пандосовский датафрейм, который затем сохраняем в БД в таблицу vacancies
df1 = pd.DataFrame({'id': IDs, 'name': names, 'description': descriptions})
 
# Тоже самое, но для таблицы skills
df2 = pd.DataFrame({'vacancy': skills_vac, 'skill': skills_name})

 
# Выводим сообщение об окончании программы
display.clear_output(wait=True)
display.display('Вакансии загружены в БД')

'Вакансии загружены в БД'

# Top skills

In [67]:
df2.rename(columns={'vacancy': 'id'}, inplace=True)

In [68]:
dfs=df2['skill'].value_counts()
dfs.to_frame().reset_index()[:11]

,index,skill
0,SQL,1138
1,Python,666
2,Git,631
3,JavaScript,486
4,Бизнес-анализ,428
5,PostgreSQL,357
6,MS SQL,353
7,MS PowerPoint,328
8,Английский язык,325
9,Java,305


In [69]:
dfs

SQL                        1138
Python                      666
Git                         631
JavaScript                  486
Бизнес-анализ               428
                           ... 
STM8                          1
Ant.design                    1
Jetpack                       1
state machine                 1
Расчет заработной платы       1
Name: skill, Length: 2349, dtype: int64

# Some work with data and expirements 

## Work for final dataset

In [70]:
df2.rename(columns={'vacancy': 'id'}, inplace=True)
df2.merge(df1)

,id,skill,name,description
0,25827004,C#,Программист C#/ .NET,<p><strong>Сфера деятельности компании Syntell...
1,25827004,Transact-SQL,Программист C#/ .NET,<p><strong>Сфера деятельности компании Syntell...
2,25827004,SQL,Программист C#/ .NET,<p><strong>Сфера деятельности компании Syntell...
3,25827004,MS SQL,Программист C#/ .NET,<p><strong>Сфера деятельности компании Syntell...
4,25827004,.NET Framework,Программист C#/ .NET,<p><strong>Сфера деятельности компании Syntell...
...,...,...,...,...
21210,45425159,Креативность,Android-developer (Sber ID),<p><strong>Sber</strong><strong> </strong><str...
21211,45425159,Тестирование,Android-developer (Sber ID),<p><strong>Sber</strong><strong> </strong><str...
21212,45425159,Unit Testing,Android-developer (Sber ID),<p><strong>Sber</strong><strong> </strong><str...
21213,45425159,Расчет зарплаты,Android-developer (Sber ID),<p><strong>Sber</strong><strong> </strong><str...


In [71]:
 df2=df2.merge(df1)
df2

,id,skill,name,description
0,25827004,C#,Программист C#/ .NET,<p><strong>Сфера деятельности компании Syntell...
1,25827004,Transact-SQL,Программист C#/ .NET,<p><strong>Сфера деятельности компании Syntell...
2,25827004,SQL,Программист C#/ .NET,<p><strong>Сфера деятельности компании Syntell...
3,25827004,MS SQL,Программист C#/ .NET,<p><strong>Сфера деятельности компании Syntell...
4,25827004,.NET Framework,Программист C#/ .NET,<p><strong>Сфера деятельности компании Syntell...
...,...,...,...,...
21210,45425159,Креативность,Android-developer (Sber ID),<p><strong>Sber</strong><strong> </strong><str...
21211,45425159,Тестирование,Android-developer (Sber ID),<p><strong>Sber</strong><strong> </strong><str...
21212,45425159,Unit Testing,Android-developer (Sber ID),<p><strong>Sber</strong><strong> </strong><str...
21213,45425159,Расчет зарплаты,Android-developer (Sber ID),<p><strong>Sber</strong><strong> </strong><str...


In [72]:
res=df2.groupby('id')['skill'].apply(list)
res

id
25827004    [C#, Transact-SQL, SQL, MS SQL, .NET Framework...
32698117                                               [Java]
33368305    [VBA, QlickView, Olap (online analytical proce...
33932334                       [MS SQL, SQL, PostgreSQL, ООП]
35546087    [Git, JavaScript, CSS, REST, HTML, React, Type...
                                  ...                        
45423584                                    [Swift, Git, iOS]
45423616                 [Java, Spring Framework, SQL, JUnit]
45424115    [Python, Linux, Git, Django Framework, SQL, Do...
45424696    [Математический анализ, Пользователь ПК, Серве...
45425159    [Креативность, Тестирование, Unit Testing, Рас...
Name: skill, Length: 3512, dtype: object

In [73]:
res=res.to_frame().reset_index()
res=res.merge(df1)

In [130]:
res[res.id=='43737395']

,id,skill,name,description
687,43737395,"[Linux, Qt, C++, Yocto, Embedded, QML, EMV, MD...",Программист C++ Qt / Software Developer (Linux),<p>Мы молодая и активно расширяющаяся команда....


# Find personal vacancies

In [127]:
def find_relevant_vacancies(personal_skills):
    relevant_vacancies_id=[]
    
    for i in range(len(myvac)):
        if (set(personal_skills).issubset(set(myvac[i]))):
            if (res.id[i] in relevant_vacancies_id):
                pass
            else:
                relevant_vacancies_id.append(res.id[i])
           
    return relevant_vacancies_id


In [131]:
perk=["C++","SQL"]
find_relevant_vacancies(perk)

['40943374',
 '41513864',
 '42397159',
 '43120104',
 '43775904',
 '44000265',
 '44010522',
 '44035419',
 '44174564',
 '44199707',
 '44620356',
 '44631776',
 '44645693',
 '44697168',
 '44708249',
 '44749259',
 '44867467',
 '45077818',
 '45081497',
 '45081753',
 '45123805',
 '45140222',
 '45183964',
 '45196785',
 '45355133']

In [94]:
relevant_vacancies_id

[]